In [ ]:
import os
import glob
import xarray as xr
import geopandas as gpd
from rasterstats import zonal_stats
import pandas as pd
from rasterio.transform import from_origin
from datetime import datetime
from joblib import Parallel, delayed

In [ ]:
import sys

os.environ['GDAL_DATA'] = os.path.join(f'{os.sep}'.join(sys.executable.split(os.sep)[:-1]), 'Library', 'share', 'gdal')

In [ ]:
# 1. Directory containing daily .nc files
nc_files_dir = "D:/SOP/India_2015-2019/press_temp"
nc_files_all = glob.glob(os.path.join(nc_files_dir, "*.nc"))

#Choosing weather forecast made at 12 PM
nc_files = sorted([x for x in nc_files_all if "12.f024.grib2.nc" in x])

# 2. Load district polygons
districts = gpd.read_file("india.geojson")

In [ ]:
# 3. Function to process a single .nc file
def process_nc_file(nc_file):
    try:
        ds = xr.open_dataset(nc_file)

        # Extract date information from filename.
        basename = os.path.basename(nc_file)
        date_str = nc_file.split(".")[2][:-2]

        # 4. Get latitude and longitude arrays from the dataset
        lat = ds['lat'].values
        lon = ds['lon'].values
        print(ds['PRES_L1'].shape)  # adjust key accordingly

        # 5. Define the affine transformation
        x_res = abs(lon[1] - lon[0])
        y_res = abs(lat[1] - lat[0])
        transform = from_origin(lon.min(), lat.max(), x_res, y_res)

        # 6. Extract the daily data for the variable (assuming it's 2D without a time dimension now)
        # If the data still has a time dimension (even as a single slice), select the first entry:
        if 'time' in ds['PRES_L1'].dims:
            daily_data = ds['PRES_L1'].isel(time=0).values
        else:
            daily_data = ds['PRES_L1'].values
        daily_data = daily_data.squeeze()
         # Ensure the data is 2D
        if daily_data.ndim != 2:
            raise ValueError(f"Invalid data shape {daily_data.shape}, expected (lat, lon)")

        # Define nodata value
        nodata_value = ds['PRES_L1'].attrs.get('_FillValue', -999)

        # Compute zonal statistics
        stats = zonal_stats(
            districts,
            daily_data,
            affine=transform,
            stats=["mean", "min", "max", "std"],
            nodata=nodata_value
        )

        for dname,district_id, stat in zip(districts['district'], districts['dt_code'], stats):
            stat.update({
                "district":dname,
                "district_id": district_id,
                "date": date_str
            })
        return stats
    except Exception as e:
        print(f"Error processing {nc_file}: {e}")
        return []

In [ ]:
# Run in parallel using all available CPU cores
results = Parallel(n_jobs=-1)(delayed(process_nc_file)(nc) for nc in nc_files)

# Flatten and save results
all_results = [entry for sublist in results for entry in sublist]
df = pd.DataFrame(all_results)
df.to_csv("daily_zonal_stats_temp_2015-2019.csv", index=False)

print("Zonal statistics saved to daily_zonal_stats_all.csv")

Zonal statistics saved to daily_zonal_stats_all.csv
